### Hands-on Lab: Build an Interactive Dashboard with Ploty Dash

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\AC\AppData\Local\Temp\ipykernel_5080\1399112611.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\AC\AppData\Local\Temp\ipykernel_5080\1399112611.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                     {'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                     ],
                                             value='ALL',
                                             placeholder='Select a Launch Site here',
                                             searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks = {    0: '0 kg',
                                                          1000: '1000 kg',
                                                          2000: '2000 kg',
                                                          3000: '3000 kg',
                                                          4000: '4000 kg',
                                                          5000: '5000 kg',
                                                          6000: '6000 kg',
                                                          7000: '7000 kg',
                                                          8000: '8000 kg',
                                                          9000: '9000 kg',
                                                          10000: '10000 kg'
                                                        },
                                                value=[min_payload, max_payload]
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Success Count for all launch sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df=spacex_df[spacex_df['Launch Site']== entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f"Total Success Launches for site {entered_site}")
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    # thought reusing filtered_df may cause issues, but tried it out of curiosity and it seems to be working fine
    
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success count on Payload mass for site {entered_site}")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [19/Feb/2023 22:55:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:41] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1668603023.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:41] "GET /_dash-component-suites/dash/deps/react@16.v2_7_0m1668603023.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:41] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_0m1668603023.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:41] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1668603023.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:41] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1668603021.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:41] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_7_0m1668603022.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:55:42] "GET /_dash-component-suites/dash

### Finding Insights Visually
Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:
1. Which site has the largest successful launches? - KSC LC-39A
2. Which site has the highest launch success rate? - KSC LC-39A
3. Which payload range(s) has the highest launch success rate? - 1952 Kg to 5300 Kg
4. Which payload range(s) has the lowest launch success rate? - 5384 Kg to 6761 Kg
5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate? - B5